<a href="https://colab.research.google.com/github/soumabkargit/Extract_Text_Image/blob/master/TesseractMethod.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Extraction du montant ttc sur les photos de facture**

#installation de l'ocr Tesseract, prise en charge de Tesseract en Anglais et installation de remotezip. 

[https://github.com/tesseract-ocr/tesseract/wiki](https://)


In [94]:
!sudo apt install tesseract-ocr
!pip install pytesseract
!sudo apt-get install tesseract-ocr-eng
!pip install remotezip

Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr is already the newest version (4.00~git2288-10f4998a-2).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr-eng is already the newest version (4.00~git24-0e00fe6-1.2).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


##Importation de toutes les dépendances

In [95]:
from matplotlib.image import imread
import numpy as np
from matplotlib import pyplot as plt
import os
import pytesseract
from pytesseract import Output
import matplotlib.patches as patches
from PIL import Image
import re
from pathlib import Path
from shutil import copyfile
from remotezip import RemoteZip
import json
import csv
# Technically not necessary in newest versions of jupyter
%matplotlib inline

## Déclarations et assignations de toutes les variables globales

In [96]:
url = 'https://expressexpense.com/large-receipt-image-dataset-SRD.zip'

lang = 'eng'

image_extract_path = "image_extract"
image_NO_extract_path = "image_NO_extract"
image_NO_read_text_path = "image_NO_read_text"
data = "data"

list_regex = []
regex_1 = r'TOTAL(.)*(\:)*(.)*(\d)'
regex_2 = r'BALANCE(.)*(\:)*(.)*(\d)'
regex_3 = r'AMOUNT(.)*(\:)*(.)*(\d)'
regex_4 = r'GRAND(.)*TOTAL(.)*(\:)*(.)*(\d)'
regex_5 = r'CASH(.)*(\:)*(.)*(\d)'
regex_6 = r'ORDER(.)*BALANCE(.)*(\:)*(.)*(\d)'
regex_amount = r'(\d)'
list_regex.append(regex_1)
list_regex.append(regex_2)
list_regex.append(regex_3)
list_regex.append(regex_4)
list_regex.append(regex_5)

good_info_candidat = 'good_info_candidat'
bad_info_candidat_value = 'bad_info_candidat_value'
bad_info_candidat_no_value = 'bad_info_candidat_no_value'
bad_info_not_foreseen = 'bad_info_not_foreseen'
warning_situation = 'warning_situation'

csv_columns = ['Amount', 'File_Name', 'File_State']
all_info_csv_file = "all_info.csv"
all_info_bad_csv_file = "all_info_bad.csv"
all_info_good_csv_file = "all_info_good.csv"

good_state = 'Good_State'
bad_state = 'Bad_State'

##Implémentation de toutes les méthodes utiles pour l'extraction du montant TTC

In [108]:
def hasNumbers(inputString):
  """Vérifier si la chaîne est alphanumérique."""
  return any(char.isdigit() for char in inputString)

def add_regex(list_regex, regex):
  """Ajouter une expression régulière."""
  list_regex.append(regex)

def extract_amount(regex_amount, string):
  """Appliquer l'expression régulière pour avoir le montant TTC en chiffre."""
  return re.search(regex_amount, string, re.IGNORECASE)

def createFolder(path):
  """Créer un dossier."""
  try:
    os.mkdir(path)
  except OSError:
    print ("la création du dossier %s a échoué" % path)
  else:
    print ("Réussite de la création du dossier %s " % path)

def extracted(url, data):
  """Téléchargement des données zipées et extraction dans un dossier"""
  try:
    with RemoteZip(url) as zip:
      zip.extractall(path=data)
  except :
    print ("Le zip n'a pas pu être chargé au lien %s " % url)
  
def is_all_values_table_null(array):
  test = False
  if all('' == s or s.isspace() for s in array):
    test = True
  return test
  
    
def extracted_text_split(imageName, lang):
  """Extraire le text sur l'image et spliter le text selon espace."""
  file_info_res = {}
  im = np.array(Image.open(imageName), dtype=np.uint8)
  try :
    extracted_text = pytesseract.image_to_string(im, lang)
  except :
    #L image n'a pas pu être traité. C'est a dire l'extraction du texte a échoué.
    copy_file_folder(data + '/' + file_name, image_NO_extract_path + '/' + file_name)
    file_info_res[bad_info_not_foreseen] = "L'extraction du texte a échoué"
    file_info_res[warning_situation] = "Erreur 001"
  text_split_line = extracted_text.splitlines()
  return text_split_line

def extracted_text_split_file_name(file_path):
  """Faire l'extraction en précisant le chemen de l'image."""
  im = file_path
  text_split_text = extracted_text_split(im, lang)
  return text_split_text

def read_all_line_file(text_split_text, file_name):
  """Rechercher les information selon les patterns défis."""
  file_info_res = {}
  if is_all_values_table_null(text_split_text):
    copy_file_folder(data + '/' + file_name, image_NO_read_text_path + '/' + file_name)
    file_info_res[bad_info_candidat_no_value] = "Nous avons une chaîne vide, Verifions s'il n'y a aucun texte sur l'image."
    file_info_res[warning_situation] = "Erreur 002"
  for line in text_split_text:
    for regex in list_regex:
      if regexLine(regex, line):
        if hasNumbers(line):
          file_info_res[good_info_candidat] = line
        else:
          file_info_res = file_info[bad_info_candidat_value] = line
          file_info_res[warning_situation] = "Erreur 003"
  return file_info_res

"""
def bad_file_info(line,file_info):

#  Donner des détaills sur les images qu'ont n'a pas pu avoir le montant TTC.

  if line != "" :
    file_info[bad_info_candidat_value] = line
  elif not line == "" :
    file_info[bad_info_candidat_no_value] = line
  else :
    file_info[bad_info_not_foreseen] = "erreur"
  return file_info
"""

def regexLine(regex, line):
  """Appliquer une expression régulière."""
  return re.search(regex, line, re.IGNORECASE)
    
def copy_file_folder(file_name, path):
  """Copier un fichier d'un dossier à un autre."""
  copyfile(file_name, path)

def get_all_names_files(data):
  """Avoir la liste de tout les fichiers."""
  try :
    list_files = os.listdir(data)
  except OSError:
    print ("La lecture de tout les fichiers dans le dossier %s a échoué" % data)
  else:
    print ("Réussite de la lecture de tout les fichiers dans le dossier %s " % data)
  return list_files

def get_all_amount(data):
  """Avoir le montant TTC de toute les photos de facture lues dans un dossier fournie."""
  files_infos_results = {}
  temp_list_files = {}

  temp_list_files = get_all_names_files(data)

  if len(temp_list_files) == 0 :
    print("Le dossier est vide")
  else :
    for image_filename in temp_list_files:
      file_info_res = get_amount(image_filename)
      files_infos_results[image_filename] = file_info_res
  return files_infos_results

def get_amount(file_name):
   """Avoir le montant TTC d'une photo de facture."""
   file_info_result = {}
   text_split_text = extracted_text_split_file_name(data + '/' + file_name)
   temp_file_info_result = read_all_line_file(text_split_text,file_name) 
   file_info_result["file_Name"] = file_name
   file_info_result['Amount'] = temp_file_info_result
   arrange_files_quality(file_info_result, file_name)
   return file_info_result

def save_data_csv(dict_data, csv_file):
  """Ajouter une information au rapport cvs """
  tab_res_info = []
  tab_res_info.append(dict_data)
  # field names 
  fields = ['Amount', 'File_Name', 'File_State'] 
  # name of csv file 
  filename = csv_file
  # writing to csv file 
  with open(filename, 'w') as csvfile: 
    # creating a csv dict writer object
    writer = csv.DictWriter(csvfile, fieldnames = fields)
    # writing headers (field names)   
    writer.writeheader()
    # writing data rows 
    writer.writerows(tab_res_info)   

def info_csv(file_name, value):
  """Construction de dictionnaire pour le rapport cvs """
  temp_csv_info_save = {}
  temp_csv_info_save['File_Name'] = file_name
  temp = value
  key_temp = list(temp.keys())
  if good_info_candidat in key_temp:
    temp_csv_info_save['Amount']= value[good_info_candidat]
    state = good_state
  elif ((bad_info_candidat_value in key_temp) ) : 
    state = bad_state 
    temp_csv_info_save['Amount']= value[bad_info_candidat_value]
  elif ( (bad_info_candidat_no_value in key_temp) ) :
    state = bad_state 
    temp_csv_info_save['Amount']= value[bad_info_candidat_no_value]
  elif ((bad_info_not_foreseen in key_temp) ) :
    state = bad_state 
    temp_csv_info_save['Amount']= value[bad_info_not_foreseen]
  else :
    state = bad_state   
    temp_csv_info_save['Amount']= "Erreur Non Prévuer"  
  temp_csv_info_save['File_State'] = state
  return temp_csv_info_save

def arrange_files_quality(file_info_res,file_name):
  """Ranger la facture selon le résultat du traitement."""
  temp_csv_info_save = {}
  value = file_info_res["Amount"]
  res_key = list(value.keys())
  print(res_key)
  if good_info_candidat in res_key:
    copyfile(data + '/' + file_name, image_extract_path + '/' + file_name) 
    temp_csv_info_save = info_csv(file_name, value )
    save_data_csv(temp_csv_info_save, all_info_good_csv_file)
    save_data_csv(temp_csv_info_save, all_info_csv_file)
  elif ( (bad_info_candidat_value in res_key) ) :  
    copyfile(data + '/' + file_name, bad_info_candidat_value + '/' + file_name)
    temp_csv_info_save = info_csv(file_name, value)
    save_data_csv(temp_csv_info_save, all_info_bad_csv_file)
    save_data_csv(temp_csv_info_save, all_info_csv_file)
  elif ( (bad_info_candidat_no_value in res_key) ) :
    copyfile(data + '/' + file_name, bad_info_candidat_no_value + '/' + file_name)
    temp_csv_info_save = info_csv(file_name, value )
    save_data_csv(temp_csv_info_save, all_info_bad_csv_file)
    save_data_csv(temp_csv_info_save, all_info_csv_file)
  elif ((bad_info_not_foreseen in res_key) ) :
    copyfile(data + '/' + file_name, bad_info_not_foreseen + '/' + file_name)
    temp_csv_info_save = info_csv(file_name, value )
    save_data_csv(temp_csv_info_save, all_info_bad_csv_file)
    save_data_csv(temp_csv_info_save, all_info_csv_file)
  else : 
    copyfile(data + '/' + file_name, warning_situation + '/' + file_name)
    temp_csv_info_save = info_csv(file_name, value )
    save_data_csv(temp_csv_info_save, all_info_bad_csv_file)
    save_data_csv(temp_csv_info_save, all_info_csv_file)


##Création des dossiers
Dossier de toute les photos de facture

Dossier des factures dont l'extraction du montant ttc a réussi

Dossier des factures dont l'extraction des montant ttc a échoué


In [ ]:
createFolder(image_extract_path)
createFolder(image_NO_extract_path)
createFolder(image_NO_read_text_path)
createFolder(bad_info_candidat_no_value)
createFolder(bad_info_candidat_value)
createFolder(bad_info_not_foreseen)
createFolder(data)
createFolder(warning_situation)

Téléchargement du zip des photos et extraction dans le dossier de toute les factures

In [10]:
extracted(url, data)

Extraction de montant ttc pour une facture

In [112]:
file_name = '1003-receipt.jpg'
res = get_amount(file_name)
print("file name :", res['file_Name'] ,"\n","Amount :", res['Amount'])

[]
file name : 1003-receipt.jpg 
 Amount : {}


Extraction de montant ttc pour toute les factures

In [105]:
dict_dat = get_all_amount(data)

Réussite de la lecture de tout les fichiers dans le dossier data 


In [106]:
dict_dat

{'1000-receipt.jpg': {'Amount': {'good_info_candidat': 'SUB TOTAL: 51.90'},
  'file_Name': '1000-receipt.jpg'},
 '1001-receipt.jpg': {'Amount': {'bad_info_candidat_no_value': "Nous avons une chaîne vide, Verifions s'il n'y a aucun texte sur l'image.",
   'warning_situation': 'Erreur 002'},
  'file_Name': '1001-receipt.jpg'},
 '1002-receipt.jpg': {'Amount': {'good_info_candidat': 'Total 161'},
  'file_Name': '1002-receipt.jpg'},
 '1003-receipt.jpg': {'Amount': {}, 'file_Name': '1003-receipt.jpg'},
 '1004-receipt.jpg': {'Amount': {'good_info_candidat': 'TOTAL. $1IS.03'},
  'file_Name': '1004-receipt.jpg'},
 '1005-receipt.jpg': {'Amount': {'good_info_candidat': 'Balance Due 179.94'},
  'file_Name': '1005-receipt.jpg'},
 '1006-receipt.jpg': {'Amount': {}, 'file_Name': '1006-receipt.jpg'},
 '1007-receipt.jpg': {'Amount': {'good_info_candidat': 'Total $143.71'},
  'file_Name': '1007-receipt.jpg'},
 '1008-receipt.jpg': {'Amount': {'good_info_candidat': 'TOTAL: $24.47'},
  'file_Name': '1008-r

In [114]:
extracted_text_split_file_name('data/1008-receipt.jpg')

['Dona Mercedes Restaurant',
 '1030 1/2 San Fernando Rd',
 'San Fernando CA 91341',
 '',
 'Vero',
 'CENTERL',
 '',
 '1 CHicharon $2.25',
 '3 Pupusa Queso $6.75',
 '1 Platanos Orden $7.75',
 '1 Diet coke $1.50',
 '2 Quesadilla salvadorena $4.00',
 '',
 'SUBTOTAL: $22.25',
 'TAX: $2.22',
 '',
 'TOTAL: $24.47',
 'TIP SUGGESTIONS',
 '18%: $4.40',
 '',
 '20%: $4.89',
 '25%: $6.12',
 '',
 'Thank You!',
 '',
 ' ',
 '']

#References

https://nanonets.com/blog/receipt-ocr/?fbclid=IwAR0H4VkHhpimqKIgEUQOUCOJS1mtk6vQREcv3MMiqJ2NpxG9LHZnsHM-aeE


https://www.datacamp.com/community/tutorials/zip-file